# 3.3- Office finder mongo version

#### Encontramos tu oficina

In [1]:
# librerias

from pymongo import MongoClient

import pandas as pd

import re
import folium

In [2]:
# conexion con Mongo

cliente=MongoClient()

db=cliente.companies

In [3]:
def data():    # funcion para realizar query a mongoDB
    query=db.companies.find({'$and':[{'$or': [{'category_code':'games_video'},
                                               {'category_code':'software'},
                                               {'category_code':'web'},
                                               {'category_code':'mobile'},
                                               {'category_code':'photo_video'},
                                               {'category_code':'ecommerce'},
                                               {'category_code':'search'},
                                               {'category_code':'network_hosting'}]},
                             {'founded_year': {'$gte':1999}},
                             {'offices': {'$ne':[]}}, 
                             {'offices.latitude': {'$ne':[]}},
                             {'offices.longitude': {'$ne':[]}},
                             {'offices.latitude': {'$ne':None}},
                             {'offices.longitude': {'$ne':None}},         
                             {'deadpooled_year':None},
                             {'number_of_employees': {'$ne':None}},
                             {'total_money_raised': {'$ne':None}}]},

                             {'name':1, '_id':0, 'offices.latitude':1, 'offices.longitude':1, 
                              'founded_year':1, 'category_code':1, 'number_of_employees':1,
                              'total_money_raised':1})


    df=pd.json_normalize(data=query, record_path='offices',
                      meta=['category_code', 'name', 'number_of_employees',
                            'founded_year', 'total_money_raised'])
    
    df.insert(2, 'geo_loc', pd.Series(list(range(len(df)))))
    df['geo_loc']=df.apply(lambda x: [x['longitude'], x['latitude']], axis=1)
    
    return df

In [4]:
data().head(10)

,latitude,longitude,geo_loc,category_code,name,number_of_employees,founded_year,total_money_raised
0,34.090368,-118.393064,"[-118.393064, 34.090368]",web,Geni,18,2006,$16.5M
1,40.746497,-74.009447,"[-74.0094471, 40.7464969]",games_video,Joost,0,2006,$45M
2,37.387845,-122.055197,"[-122.055197, 37.387845]",web,Plaxo,50,2002,$28.3M
3,37.778613,-122.395289,"[-122.395289, 37.778613]",search,Powerset,60,2006,$22.5M
4,34.017606,-118.487267,"[-118.487267, 34.017606]",web,Mahalo,40,2007,$21M
5,37.788482,-122.409173,"[-122.409173, 37.788482]",games_video,Kyte,40,2006,$23.4M
6,37.480999,-122.173887,"[-122.173887, 37.480999]",mobile,Jingle Networks,35,2005,$88.7M
7,40.726040,-73.995722,"[-73.995722, 40.72604]",web,Meetup,75,2002,$18.3M
8,37.390056,-122.067692,"[-122.067692, 37.390056]",photo_video,SmugMug,62,2002,$0
9,37.423390,-122.089951,"[-122.0899512, 37.4233896]",mobile,Jajah,110,2005,$33M


In [5]:
def exchange(df):
    res=[]
    number=1
    symbol=1
    
    for e in df:
        if e[-1]=='k': number=1e3
        elif e[-1]=='M': number=1e6
        elif e[-1]=='B': number=1e9
            
        if e[0]=='£': symbol=1.3
        elif e[0]=='€' : symbol=1.12
            
        n_df=re.findall('[0-9.]+', e)
        
        res.append(int(float(n_df[0])*number*symbol))
        
    return res

In [6]:
def classifying(date, employees):
    
    company_class=[]
    
    for i in range(len(date)):
        
        if date[i]>=2008 and employees[i]<100:
            company_class.append('StartUp')
            
        else:
            company_class.append('Big')
            
    return company_class

In [7]:
# llamada a los datos

df=data()

print('numero total de compañias consideras:', len(df))

numero total de compañias consideras: 2881


In [8]:
df['n_total_money']=exchange(df.total_money_raised)

df['company_class']=classifying(df.founded_year, df.number_of_employees)

df.head()

,latitude,longitude,geo_loc,category_code,name,number_of_employees,founded_year,total_money_raised,n_total_money,company_class
0,34.090368,-118.393064,"[-118.393064, 34.090368]",web,Geni,18,2006,$16.5M,16500000,Big
1,40.746497,-74.009447,"[-74.0094471, 40.7464969]",games_video,Joost,0,2006,$45M,45000000,Big
2,37.387845,-122.055197,"[-122.055197, 37.387845]",web,Plaxo,50,2002,$28.3M,28300000,Big
3,37.778613,-122.395289,"[-122.395289, 37.778613]",search,Powerset,60,2006,$22.5M,22500000,Big
4,34.017606,-118.487267,"[-118.487267, 34.017606]",web,Mahalo,40,2007,$21M,21000000,Big


In [9]:
# insertar en Mongo

# df.to_json('geoloc.json', orient='records', lines=True)

In [10]:
db.geoloc.insert_many(df.to_dict('records'))

In [11]:
# crear indice 2D

db.geoloc.create_index([('geo_loc', '2dsphere')])

'geo_loc_2dsphere'

In [12]:
# geolocalizacion

def geoquery(df):
    
    list_money_mean=[]
    sb_ratio=[]
    n_companies=[]
    
    for i in range(len(df)):
        
        q=list(
            db.geoloc.find(
                {'geo_loc': {'$near':{'$geometry':{
                    'type':'Point',
                    'coordinates':[df.longitude[i], df.latitude[i]]
                },
                                     '$maxDistance':1000}}}
            )
        )
        
        l=len(q)
        
        money_mean=sum([q[j]['n_total_money'] for j in range(l)])/l
        
        list_money_mean.append(money_mean)
        
        big=sum([q[k]['company_class'].count('Big') if q[k]['company_class'].count('Big')!=0 else 1 for k in range(l)])
        start=sum([q[k]['company_class'].count('StartUp') for k in range(l)])
        
        sb_ratio.append(start/big)
        
        n_companies.append(l)
        
    df['around_companies']=n_companies

    df['money_density']=list_money_mean

    df['sb_ratio']=sb_ratio
        
    return df

In [13]:
def geomap(df, zoom_start=1.5):

    df.index=range(len(df)) 

    mapa=folium.Map(location=[0, 0],
                tiles='openstreetmap', zoom_start=zoom_start)

    for i in range(1, len(df)):

        folium.CircleMarker([float(df.latitude[i]), float(df.longitude[i])], popup=str(df.name[i]), 
                             radius=1, icon=folium.Icon()).add_to(mapa)

    mapa.save('images/geomap.html')
    
    
    return mapa

In [14]:
df=geoquery(df)

df.head()

,latitude,longitude,geo_loc,category_code,name,number_of_employees,founded_year,total_money_raised,n_total_money,company_class,around_companies,money_density,sb_ratio
0,34.090368,-118.393064,"[-118.393064, 34.090368]",web,Geni,18,2006,$16.5M,16500000,Big,1,1.650000e+07,0.000000
1,40.746497,-74.009447,"[-74.0094471, 40.7464969]",games_video,Joost,0,2006,$45M,45000000,Big,8,7.325250e+06,0.375000
2,37.387845,-122.055197,"[-122.055197, 37.387845]",web,Plaxo,50,2002,$28.3M,28300000,Big,2,2.715000e+07,0.000000
3,37.778613,-122.395289,"[-122.395289, 37.778613]",search,Powerset,60,2006,$22.5M,22500000,Big,27,1.105724e+07,0.222222
4,34.017606,-118.487267,"[-118.487267, 34.017606]",web,Mahalo,40,2007,$21M,21000000,Big,3,8.733333e+06,0.333333


In [15]:
df.describe()

,latitude,longitude,n_total_money,around_companies,money_density,sb_ratio
count,2881.000000,2881.000000,2.881000e+03,2881.000000,2.881000e+03,2881.000000
mean,38.052028,-60.793188,4.921155e+06,6.052412,4.908734e+06,0.281803
std,15.876878,66.289707,2.141485e+07,11.280223,1.455534e+07,0.365205
min,-41.565586,-159.480262,0.000000e+00,1.000000,0.000000e+00,0.000000
25%,35.911733,-118.382327,0.000000e+00,1.000000,0.000000e+00,0.000000
50%,40.016582,-79.420926,0.000000e+00,2.000000,0.000000e+00,0.100000
75%,45.629498,-0.427418,1.120000e+05,5.000000,4.395730e+06,0.500000
max,65.056601,175.260400,4.090000e+08,75.000000,2.526000e+08,1.000000


In [16]:
print('oficinas.')

mapa=geomap(df.loc[:1500])
mapa

oficinas.


In [17]:
print ('Compañias alrededor=10, sb_ratio>0.45 y densidad monetaria>1.000.000$')
mapa2=geomap(df[(df['around_companies']>=10) & (df['sb_ratio']>0.45) & (df['money_density']>1e6)])
display (mapa2)

Compañias alrededor=10, sb_ratio>0.45 y densidad monetaria>1.000.000$


In [18]:
print ('Compañias alrededor>=70, sb_ratio>0.25 y densidad monetaria>10.000.000$')
mapa3=geomap(df[(df['around_companies']>=70) & (df['sb_ratio']>0.25) & (df['money_density']>1e7)])
display (mapa3)

Compañias alrededor>=70, sb_ratio>0.25 y densidad monetaria>10.000.000$
